In [1]:
import torch
from transformers import BertTokenizer
from BertRegressor import BertRegressor

c:\Users\Nemsun\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the model architecture and load the trained weights

model = BertRegressor()
model.load_state_dict(torch.load('bert_model.pt'))
model.eval()

BertRegressor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [3]:
# Load the tokenizer and tokenize the input text

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

playlist_name = "chill rab mix"
encoded_playlist_name = tokenizer(text = playlist_name, 
                           add_special_tokens=True,
                           padding = 'max_length',
                           truncation = 'longest_first',
                           max_length = 300,
                           return_attention_mask = True)

input_ids = encoded_playlist_name['input_ids']
attention_masks = encoded_playlist_name['attention_mask']

In [4]:
# run the model on the input text
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"using device: {device}")

with torch.no_grad():
    output = model(torch.tensor(input_ids).to(device).unsqueeze(0), 
                   torch.tensor(attention_masks).to(device).unsqueeze(0))
    
    print(f"predicted playlist score: {output.tolist()}")

using device: cuda
predicted playlist score: [[0.38439199328422546, 0.3246513307094574, -0.129065603017807, -0.6882642507553101, -0.04229501262307167, 0.10190823674201965, -0.5402419567108154, -0.17356836795806885, -1.2401353120803833, 0.4679490923881531, -0.8906165361404419, 0.11346378922462463, -0.7855994701385498]]


In [6]:
# define functions that have more generatlized use

import torch
from torch.utils.data import TensorDataset, DataLoader

def predict(model, dataloader, device):
    model.eval()
    output = []
    for batch in dataloader:
        batch_inputs, batch_masks = \
                                  tuple(b.to(device) for b in batch)
        with torch.no_grad():
            output += model(batch_inputs, 
                            batch_masks).tolist()
    return output


def create_dataloaders(inputs, masks, batch_size=13):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    dataset = TensorDataset(input_tensor, mask_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return dataloader


def prepare_inputs(playlist_names, tokenizer):
    encoded_playlist_names = tokenizer(text = playlist_names, 
                                       add_special_tokens=True,
                                       padding = 'max_length',
                                       truncation = 'longest_first',
                                       max_length = 300,
                                       return_attention_mask = True)
    
    input_ids = encoded_playlist_names['input_ids']
    attention_masks = encoded_playlist_names['attention_mask']
    
    return input_ids, attention_masks

In [7]:
songs = ["chill rap mix", "coffee shop", "aggressive rap and drill"]
input_ids, attention_masks = prepare_inputs(songs, tokenizer)
dataloader = create_dataloaders(input_ids, attention_masks)
prediction = predict(model, dataloader, device)

print("predicted playlist scores: ")
for idx, song in enumerate(songs):
    print(f"{song}: {prediction[idx]}")


predicted playlist scores: 
chill rap mix: [-0.13857638835906982, 0.8016530275344849, -0.04099654033780098, -0.3614689111709595, -0.3748628795146942, -0.00996387843042612, 0.02088908851146698, 0.07665769755840302, -1.0320204496383667, 1.6485298871994019, -0.17494386434555054, 0.4433545768260956, -0.7561355233192444]
coffee shop: [1.4801605939865112, -0.041393179446458817, -0.6346140503883362, -1.6483286619186401, 1.4928240776062012, -0.20013530552387238, -1.2966294288635254, -1.089979887008667, -0.2973755896091461, -0.586452305316925, -1.0738660097122192, -0.28506624698638916, -0.4973054528236389]
aggressive rap and drill: [-0.5884665250778198, 1.2944731712341309, -0.5323115587234497, 0.1520185023546219, -0.6506642699241638, -0.29961204528808594, 0.6092473268508911, 0.5144625902175903, -0.21855659782886505, 2.3652360439300537, 0.8908799290657043, 0.7016943693161011, -0.0943540707230568]


In [8]:
songs = ["coffee shop"]
input_ids, attention_masks = prepare_inputs(songs, tokenizer)
dataloader = create_dataloaders(input_ids, attention_masks)

print("predicted playlist scores: ")
for idx, song in enumerate(songs):
    print(f"{song}: {prediction[idx]}")

predicted playlist scores: 
coffee shop: [-0.13857638835906982, 0.8016530275344849, -0.04099654033780098, -0.3614689111709595, -0.3748628795146942, -0.00996387843042612, 0.02088908851146698, 0.07665769755840302, -1.0320204496383667, 1.6485298871994019, -0.17494386434555054, 0.4433545768260956, -0.7561355233192444]
